In [1]:
import ROOT
import numpy as np
from itertools import combinations, combinations_with_replacement, product

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x7320a40


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
# plot_path = "plots/angles/mg5_1M"
plot_path = "/dev/null"
oo_plot_name = "OO_wz.pdf"

In [5]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [6]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")
# ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools_old.h\"")
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [7]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882
# or use Markus old values, whatever the file was generated with obviously
# mW = 80.22
# mZ = 91.17

# ROOT.OOTools.init(250, mW, mZ)
# ROOT.OOTools.init(240, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

cw2 = (mW/mZ)**2
sw2 = 1 - cw2

In [8]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [9]:
%%cpp
using namespace ROOT::VecOps;

In [10]:
# df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_190/ww_ac_e-1.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/gen/whizard_sw_semi_250/ww.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/gen/whizard_sw_semi_190/ww.edm4hep.root")
df = ROOT.RDataFrame("events", "data/gen/whizard_sw_forced_ww_semi_190/ww.edm4hep.root")
# n_events = 50000
# n_events = 100000
n_events = 1000000
df = df.Range(n_events)

In [11]:

df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")
df = df.Define("weight", "(double) Parameters.get<float>(\"_weight\").value()")
df = df.Define("weight_alt1", "Parameters.get<double>(\"weight_alt1\").value()")
df = df.Define("weight_alt2", "Parameters.get<double>(\"weight_alt2\").value()")
df = df.Define("weight_alt3", "Parameters.get<double>(\"weight_alt3\").value()")
df = df.Define("sqme", "Parameters.get<double>(\"sqme\").value()")
# FIXME: debug only lol
# df = df.Define("weight_alt1", "1.0")
# df = df.Define("weight_alt2", "1.0")
# df = df.Define("weight_alt3", "1.0")

In [12]:
# df = df.Vary("weight", "RVec<double>{weight * weight_alt1, weight * weight_alt2, weight * weight_alt3}", ["g1z_up", "ka_up", "la_up"], "aTGC")

In [13]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("e_MC_idx", "ArgMax(MCParticles.PDG == 11 && MCParticles.generatorStatus == 1)")
df = df.Define("enu_MC_idx", "ArgMax(MCParticles.PDG == -12 && MCParticles.generatorStatus == 1)")
# only named p and pnu out of historical reasons, in reality these are the quarks
# take the anti-fermion here
df = df.Define("p_MC_idx", "ArgMax(MCParticles.PDG == -1 && MCParticles.generatorStatus == 1)")
df = df.Define("pnu_MC_idx", "ArgMax(MCParticles.PDG == 2 && MCParticles.generatorStatus == 1)")
df = df.Define("Wp_MC_idx", "ArgMax(MCParticles.PDG == 24)")
df = df.Define("Wm_MC_idx", "ArgMax(MCParticles.PDG == -24)")

df = df.Define("e_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[e_MC_idx], MCParticles.momentum.y[e_MC_idx], MCParticles.momentum.z[e_MC_idx], MCParticles.mass[e_MC_idx])")
df = df.Define("enu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[enu_MC_idx], MCParticles.momentum.y[enu_MC_idx], MCParticles.momentum.z[enu_MC_idx], MCParticles.mass[enu_MC_idx])")
df = df.Define("p_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[p_MC_idx], MCParticles.momentum.y[p_MC_idx], MCParticles.momentum.z[p_MC_idx], MCParticles.mass[p_MC_idx])")
df = df.Define("pnu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[pnu_MC_idx], MCParticles.momentum.y[pnu_MC_idx], MCParticles.momentum.z[pnu_MC_idx], MCParticles.mass[pnu_MC_idx])")
df = df.Define("Wp_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wp_MC_idx], MCParticles.momentum.y[Wp_MC_idx], MCParticles.momentum.z[Wp_MC_idx], MCParticles.mass[Wp_MC_idx])")
df = df.Define("Wm_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wm_MC_idx], MCParticles.momentum.y[Wm_MC_idx], MCParticles.momentum.z[Wm_MC_idx], MCParticles.mass[Wm_MC_idx])")

# XXX: comment in for sw files
df = df.Redefine("Wp_MC_lvec", "p_MC_lvec + pnu_MC_lvec")
df = df.Redefine("Wm_MC_lvec", "e_MC_lvec + enu_MC_lvec")

In [14]:

df = df.Define("l_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "e_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "e_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("e_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("enu_MC_lvec_e", "enu_MC_lvec.energy()")
df = df.Define("p_MC_lvec_e", "p_MC_lvec.energy()")
df = df.Define("pnu_MC_lvec_e", "pnu_MC_lvec.energy()")

df = df.Define("e_MC_lvec_px", "e_MC_lvec.px()")
df = df.Define("e_MC_lvec_py", "e_MC_lvec.py()")
df = df.Define("e_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("p_MC_lvec_px", "p_MC_lvec.px()")
df = df.Define("p_MC_lvec_py", "p_MC_lvec.py()")
df = df.Define("p_MC_lvec_pz", "p_MC_lvec.pz()")

df = df.Define("Wm_MC_lvec_px", "Wm_MC_lvec.px()")
df = df.Define("Wm_MC_lvec_py", "Wm_MC_lvec.py()")
df = df.Define("Wm_MC_lvec_pz", "Wm_MC_lvec.pz()")

df = df.Define("Wp_MC_lvec_px", "Wp_MC_lvec.px()")
df = df.Define("Wp_MC_lvec_py", "Wp_MC_lvec.py()")
df = df.Define("Wp_MC_lvec_pz", "Wp_MC_lvec.pz()")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

df = df.Define("W_minus_lvec", "Wm_MC_lvec")
df = df.Define("W_plus_lvec", "Wp_MC_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, -1)")

df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")
# FIXME: e_minus_vec?
df = df.Define("CosTheta_W_plus", "ROOT::Math::VectorUtil::CosTheta(W_plus_lvec, e_minus_vec)")


In [15]:
df = df.Define("enu_m", "(e_MC_lvec + enu_MC_lvec).M()")
df = df.Define("pnu_m", "(p_MC_lvec + pnu_MC_lvec).M()")

In [16]:
# df = df.Define("e_transformed_lvec", "WWTools::starVectorOO(W_minus_lvec, e_MC_lvec)")
# df = df.Define("p_transformed_lvec", "WWTools::starVectorOO(W_plus_lvec, p_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec, {0, 0, 1})")
# df = df.Define("p_transformed_lvec", "WWTools::starVector(W_plus_lvec, p_MC_lvec)")
df = df.Define("e_transformed_lvec", "WWTools::starVectorHagiwara(W_minus_lvec, e_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("p_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, p_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("pnu_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, pnu_MC_lvec, {0, 0, 1}, W_minus_lvec)")

In [17]:

df = df.Define("e_theta_prime", "e_transformed_lvec.Theta()")
df = df.Define("e_phi_prime", "e_transformed_lvec.Phi()")
# df = df.Define("e_phi_prime", "e_transformed_lvec.Phi() + ROOT::Math::Pi()")
df = df.Define("p_theta_prime", "p_transformed_lvec.Theta()")
df = df.Define("p_phi_prime", "p_transformed_lvec.Phi()")
# df = df.Define("p_phi_prime", "p_transformed_lvec.Phi() + ROOT::Math::Pi()")
# df = df.Define("l_theta_prime", "auto theta = l_transformed_lvec.Theta(); return l_MC_charge == 1. ? ROOT::Math::Pi() - theta : theta;")
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); return l_MC_charge == 1. ? phi - ROOT::Math::Pi() : phi;")

df = df.Define("pnu_theta_prime", "pnu_transformed_lvec.Theta()")
df = df.Define("pnu_phi_prime", "pnu_transformed_lvec.Phi()")

# OPAL shift W plus by 180 deg, but this gives it the wrong shape?!
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); auto phi_s = l_MC_charge == 1. ? phi + ROOT::Math::Pi() : phi; return phi_s > ROOT::Math::Pi() ? phi_s - 2 * ROOT::Math::Pi() : phi_s;")
# df = df.Define("p_phi_prime", "auto phi = p_transformed_lvec.Phi() + ROOT::Math::Pi(); return phi > ROOT::Math::Pi() ? phi - 2 * ROOT::Math::Pi() : phi;")

df = df.Define("e_cosTheta_prime", "cos(e_theta_prime)")
df = df.Define("p_cosTheta_prime", "cos(p_theta_prime)")
df = df.Define("pnu_cosTheta_prime", "cos(pnu_theta_prime)")
# df = df.Define("p_cosTheta_prime", "-cos(p_theta_prime)")

In [18]:
for var in ["aw", "awph", "abph", "dg1z", "dkg", "lz"] + [f"fg{i}" for i in range(1,8)] + [f"fz{i}" for i in range(1,8)]:
    df = df.Define(f"{var}_jet1", f"OOTools::O_{var}(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
    df = df.Define(f"{var}_jet2", f"OOTools::O_{var}(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
    # FIXME this is of course not the same as taking both values and weighting them by 0.5... :(
    # df = df.Define(var, f"({var}_jet1 + {var}_jet2) * 0.5")
    # Diehl paper says its also ok to take the leading one if the OO (S1_jet1 + S1_jet2) / (S0_jet1 + S0_jet2)
    # which it effectively is due to the -co2, ph2 + Pi term
    # so I can just choose one of the two however I want as both "should" return the same
    # at least on generator level with no magic and only CC03...
    df = df.Define(var, f"{var}_jet1")
df = df.Define("proba_sm", "OOTools::proba_sm(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# should ideally be equal, but will probably rather be a constant factor...
df = df.Define("sqme_over_proba_sm", "sqme / proba_sm")
df = df.Define("inv_proba_sm", "1 / proba_sm")
df = df.Define("inv_sqme", "1 / sqme")

# df = df.Redefine("weight", "weight * inv_sqme")
df = df.Redefine("weight", "weight * inv_proba_sm")
df = df.Define("w_inv_sqme", "weight * inv_sqme")
df = df.Define("w_inv_proba", "weight * inv_proba_sm")
# to have consistent naming with Markus
df = df.Define("co", "CosTheta_W_minus")
df = df.Define("co1", "e_cosTheta_prime")
df = df.Define("co2", "p_cosTheta_prime")
df = df.Define("ph1", "e_phi_prime")
df = df.Define("ph2", "p_phi_prime")

# FIXME: quick and dirty test
# df = df.Redefine(f"dg1z", f"OOTools::dg1z_new(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Redefine(f"dkg", f"OOTools::dkg_new(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Redefine(f"lz", f"OOTools::lz_new(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

In [19]:
df = df.Define("O1", "dg1z")
df = df.Define("O2", "dkg")
df = df.Define("O3", "lz")
# for i, j in combinations_with_replacement((1, 2, 3), r=2):
for i, j in product((1, 2, 3), repeat=2):
    df = df.Define(f"O{i}O{j}", f"O{i} * O{j}")

In [20]:
# FIXME: for debugging
# df = df.Define("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
# df = df.Filter(f"abs(enu_m - {mW}) < 2.5 && abs(pnu_m - {mW}) < 2.5")
df = df.Filter(f"abs(enu_m - {mW}) < 1.25 && abs(pnu_m - {mW}) < 1.25")

In [21]:
oo_bins = 100
# oo_bins = 25
# oo_bins = 250
oo_start = -4.
oo_end = 2.

h_aw = df.Histo1D(("", ";O_{#alphaW}", oo_bins, oo_start, oo_end), "aw", "weight")
h_awph = df.Histo1D(("", ";O_{#alphaW#phi}", oo_bins, oo_start, oo_end), "awph", "weight")
h_abph = df.Histo1D(("", ";O_{#alphaB#phi}", oo_bins, oo_start, oo_end), "abph", "weight")

h_dg1z = df.Histo1D(("", ";O_{#delta g_{1}^{Z}}", oo_bins, oo_start, oo_end), "dg1z", "weight")
h_dkg = df.Histo1D(("", ";O_{#delta#kappa_{#gamma}}", oo_bins, oo_start, oo_end), "dkg", "weight")
h_lz = df.Histo1D(("", ";O_{#lambda_{Z}}", oo_bins, oo_start, oo_end), "lz", "weight")

h_fg1 = df.Histo1D(("", ";O_{f_{1}^{#gamma}}", oo_bins, oo_start, oo_end), "fg1", "weight")
h_fg2 = df.Histo1D(("", ";O_{f_{2}^{#gamma}}", oo_bins, oo_start, oo_end), "fg2", "weight")
h_fg3 = df.Histo1D(("", ";O_{f_{3}^{#gamma}}", oo_bins, oo_start, oo_end), "fg3", "weight")
h_fg4 = df.Histo1D(("", ";O_{f_{4}^{#gamma}}", oo_bins, oo_start, oo_end), "fg4", "weight")
h_fg5 = df.Histo1D(("", ";O_{f_{5}^{#gamma}}", oo_bins, oo_start, oo_end), "fg5", "weight")
h_fg6 = df.Histo1D(("", ";O_{f_{6}^{#gamma}}", oo_bins, oo_start, oo_end), "fg6", "weight")
h_fg7 = df.Histo1D(("", ";O_{f_{7}^{#gamma}}", oo_bins, oo_start, oo_end), "fg7", "weight")

h_fz1 = df.Histo1D(("", ";O_{f_{1}^{Z}}", oo_bins, oo_start, oo_end), "fz1", "weight")
h_fz2 = df.Histo1D(("", ";O_{f_{2}^{Z}}", oo_bins, oo_start, oo_end), "fz2", "weight")
h_fz3 = df.Histo1D(("", ";O_{f_{3}^{Z}}", oo_bins, oo_start, oo_end), "fz3", "weight")
h_fz4 = df.Histo1D(("", ";O_{f_{4}^{Z}}", oo_bins, oo_start, oo_end), "fz4", "weight")
h_fz5 = df.Histo1D(("", ";O_{f_{5}^{Z}}", oo_bins, oo_start, oo_end), "fz5", "weight")
h_fz6 = df.Histo1D(("", ";O_{f_{6}^{Z}}", oo_bins, oo_start, oo_end), "fz6", "weight")
h_fz7 = df.Histo1D(("", ";O_{f_{7}^{Z}}", oo_bins, oo_start, oo_end), "fz7", "weight")

h_sqme = df.Histo1D("sqme")
h_proba_sm = df.Histo1D("proba_sm")
h_sqme_over_proba_sm = df.Histo1D(("", ";sqme_over_proba_sm", 100, 0., 5e-5), "sqme_over_proba_sm")
h_2d_sqme_proba_sm = df.Histo2D(("", ";sqme;proba_sm", 100, 0., 0.002, 500, 0., 100.), "sqme", "proba_sm")

In [22]:
v_aw = ROOT.RDF.Experimental.VariationsFor(h_aw)
v_awph = ROOT.RDF.Experimental.VariationsFor(h_awph)
v_abph = ROOT.RDF.Experimental.VariationsFor(h_abph)

v_dg1z = ROOT.RDF.Experimental.VariationsFor(h_dg1z)
v_dkg = ROOT.RDF.Experimental.VariationsFor(h_dkg)
v_lz = ROOT.RDF.Experimental.VariationsFor(h_lz)

v_fg1 = ROOT.RDF.Experimental.VariationsFor(h_fg1)
v_fg2 = ROOT.RDF.Experimental.VariationsFor(h_fg2)
v_fg3 = ROOT.RDF.Experimental.VariationsFor(h_fg3)
v_fg4 = ROOT.RDF.Experimental.VariationsFor(h_fg4)
v_fg5 = ROOT.RDF.Experimental.VariationsFor(h_fg5)
v_fg6 = ROOT.RDF.Experimental.VariationsFor(h_fg6)
v_fg7 = ROOT.RDF.Experimental.VariationsFor(h_fg7)

v_fz1 = ROOT.RDF.Experimental.VariationsFor(h_fz1)
v_fz2 = ROOT.RDF.Experimental.VariationsFor(h_fz2)
v_fz3 = ROOT.RDF.Experimental.VariationsFor(h_fz3)
v_fz4 = ROOT.RDF.Experimental.VariationsFor(h_fz4)
v_fz5 = ROOT.RDF.Experimental.VariationsFor(h_fz5)
v_fz6 = ROOT.RDF.Experimental.VariationsFor(h_fz6)
v_fz7 = ROOT.RDF.Experimental.VariationsFor(h_fz7)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xd7f1460


In [23]:
h_enu_m = df.Histo1D("enu_m")
h_pnu_m = df.Histo1D("pnu_m")

In [24]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "enu_MC_lvec_e", "weight")
h_2d_e_e = df.Histo2D(("", ";E_{e-} [GeV];E_{#bar{#nu}} [GeV]", 300, 0., 150., 300, 0., 150.), "e_MC_lvec_e", "enu_MC_lvec_e", "weight")
h_2d_p_e = df.Histo2D(("", ";E_{e+} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "p_MC_lvec_e", "pnu_MC_lvec_e", "weight")
h_2d_lnu_pzpt = df.Histo2D(("", ";P_{z}(e) [GeV];P_{t}(e) [GeV]", 150, -150., 150., 150, 0., 150.), "l_MC_lvec_pz", "l_MC_lvec_pt", "weight")
h_2d_lnu_cosThetapt = df.Histo2D(("", ";cos #theta(e);P_{t}(e) [GeV]", 150, -1., 1., 150, 0., 150.), "l_MC_lvec_cosTheta", "l_MC_lvec_pt", "weight")

h_2d_ep_px = df.Histo2D(("", ";P_{x}(e-) [GeV];P_{x}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_px", "p_MC_lvec_px", "weight")
h_2d_ep_py = df.Histo2D(("", ";P_{y}(e-) [GeV];P_{y}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_py", "p_MC_lvec_py", "weight")
h_2d_ep_pz = df.Histo2D(("", ";P_{z}(e-) [GeV];P_{z}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_pz", "p_MC_lvec_pz", "weight")

h_2d_WW_px = df.Histo2D(("", ";P_{x}(W-) [GeV];P_{x}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_px", "Wp_MC_lvec_px", "weight")
h_2d_WW_py = df.Histo2D(("", ";P_{y}(W-) [GeV];P_{y}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_py", "Wp_MC_lvec_py", "weight")
h_2d_WW_pz = df.Histo2D(("", ";P_{z}(W-) [GeV];P_{z}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_pz", "Wp_MC_lvec_pz", "weight")

h_e_px = df.Histo1D(("", ";P_{x}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_px", "weight")
h_e_py = df.Histo1D(("", ";P_{y}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_py", "weight")
h_e_pz = df.Histo1D(("", ";P_{z}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_pz", "weight")

h_p_px = df.Histo1D(("", ";P_{x}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_px", "weight")
h_p_py = df.Histo1D(("", ";P_{y}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_py", "weight")
h_p_pz = df.Histo1D(("", ";P_{z}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_pz", "weight")

h_CosTheta_W_minus = df.Histo1D(("", ";cos #theta_{W^{-}}", 40, -1., 1.), "CosTheta_W_minus", "weight")
h_CosTheta_W_plus = df.Histo1D(("", ";cos #theta_{W^{+}}", 40, -1., 1.), "CosTheta_W_plus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_plus_theta_prime = df.Histo1D("p_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "p_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "p_phi_prime", "weight")
h_l_minus_theta_prime = df.Histo1D("e_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{-}}", 40, -1., 1.), "e_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "e_phi_prime", "weight")

h_l_cosTheta_prime = h_l_plus_cosTheta_prime.Clone()
h_l_cosTheta_prime.Add(h_l_minus_cosTheta_prime.GetPtr())
h_l_cosTheta_prime.SetTitle(";cos #theta*_{e^{#pm}}")
h_l_phi_prime = h_l_plus_phi_prime.Clone()
h_l_phi_prime.Add(h_l_minus_phi_prime.GetPtr())
h_l_phi_prime.SetTitle(";#phi*_{e#pm}")

In [25]:
# v_CosTheta_W_minus = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus)
# v_l_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_cosTheta_prime)
# v_l_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_phi_prime)
v_l_plus_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_plus_cosTheta_prime)
v_l_plus_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_prime)
v_l_minus_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_minus_cosTheta_prime)
v_l_minus_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_minus_phi_prime)

In [26]:
pi = ROOT.Math.Pi()
nbins = 40

h_2D = {}
from itertools import combinations
angles = ["co", "co1", "co2", "ph1", "ph2"]
for a, b in combinations(angles, 2):
    k = f"{a}_{b}"
    bin_end_a = pi if a.startswith("ph") else 1.
    bin_end_b = pi if b.startswith("ph") else 1.
    h_model = ("", f";{a};{b}", nbins, -bin_end_a, bin_end_a, nbins, -bin_end_b, bin_end_b)
    h_2D[k] = df.Histo2D(h_model, a, b, "weight")

In [27]:
f_vars = ["fg1", "fz1", "fg2", "fz2", "fg3", "fz3", "fg5", "fz5"]
# vars = ["fg1", "fg2", "fg3"]
tgc_vars = ["dg1z", "dkg", "lz"]
weights = ["weight_alt1", "weight_alt2", "weight_alt3"]
combs = [f"O{i}O{j}" for i, j in product((1, 2, 3), repeat=2)]

# columns = df.AsNumpy(f_vars + tgc_vars + weights + combs)

In [28]:
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(ROOT.kBird)
ROOT.gStyle.SetNumberContours(20)
c_2D = {}
for a, b in combinations(angles, 2):
    k = f"{a}_{b}"
    c_2D[k] = ROOT.TCanvas()
    h_2D[k].Draw("colz0")
    c_2D[k].Draw()

In [29]:
ROOT.gStyle.SetOptStat(1)
c_sqme = ROOT.TCanvas()
h_sqme.Draw()
c_sqme.Draw()

c_proba_sm = ROOT.TCanvas()
h_proba_sm.Draw()
c_proba_sm.Draw()

c_sqme_over_proba_sm = ROOT.TCanvas()
h_sqme_over_proba_sm.Draw()
c_sqme_over_proba_sm.Draw()

c_2d_sqme_proba_sm = ROOT.TCanvas()
h_2d_sqme_proba_sm.Draw()
c_2d_sqme_proba_sm.Draw()

In [30]:
c_enu_m = ROOT.TCanvas()
h_enu_m.Draw()
c_enu_m.Draw()

c_pnu_m = ROOT.TCanvas()
h_pnu_m.Draw()
c_pnu_m.Draw()

In [31]:
c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.SetMinimum(0)
h_CosTheta_W_minus.Draw()
# v_CosTheta_W_minus["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_CosTheta_W_minus["aTGC:g1z_up"].Draw("same")
# v_CosTheta_W_minus["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_CosTheta_W_minus["aTGC:ka_up"].Draw("same")
# v_CosTheta_W_minus["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_CosTheta_W_minus["aTGC:la_up"].Draw("same")
c_CosTheta_W_minus.Draw()

# c_l_cosTheta_prime = ROOT.TCanvas()
# h_l_cosTheta_prime.SetMinimum(0)
# h_l_cosTheta_prime.Draw()
# # v_l_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# # v_l_cosTheta_prime["aTGC:g1z_up"].Draw("same")
# # v_l_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# # v_l_cosTheta_prime["aTGC:ka_up"].Draw("same")
# # v_l_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# # v_l_cosTheta_prime["aTGC:la_up"].Draw("same")
# c_l_cosTheta_prime.Draw()

# c_l_phi_prime = ROOT.TCanvas()
# h_l_phi_prime.SetMinimum(0)
# h_l_phi_prime.Draw()
# # v_l_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# # v_l_phi_prime["aTGC:g1z_up"].Draw("same")
# # v_l_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# # v_l_phi_prime["aTGC:ka_up"].Draw("same")
# # v_l_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# # v_l_phi_prime["aTGC:la_up"].Draw("same")
# c_l_phi_prime.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.SetMinimum(0)
h_l_minus_cosTheta_prime.Draw()
# v_l_minus_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_minus_cosTheta_prime["aTGC:g1z_up"].Draw("same")
# v_l_minus_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_minus_cosTheta_prime["aTGC:ka_up"].Draw("same")
# v_l_minus_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_minus_cosTheta_prime["aTGC:la_up"].Draw("same")
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.Draw()
# v_l_minus_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_minus_phi_prime["aTGC:g1z_up"].Draw("same")
# v_l_minus_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_minus_phi_prime["aTGC:ka_up"].Draw("same")
# v_l_minus_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_minus_phi_prime["aTGC:la_up"].Draw("same")
c_l_minus_phi_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.SetMinimum(0)
h_l_plus_cosTheta_prime.Draw()
# v_l_plus_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_plus_cosTheta_prime["aTGC:g1z_up"].Draw("same")
# v_l_plus_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_plus_cosTheta_prime["aTGC:ka_up"].Draw("same")
# v_l_plus_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_plus_cosTheta_prime["aTGC:la_up"].Draw("same")
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.SetMinimum(0)
h_l_plus_phi_prime.Draw()
# v_l_plus_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_plus_phi_prime["aTGC:g1z_up"].Draw("same")
# v_l_plus_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_plus_phi_prime["aTGC:ka_up"].Draw("same")
# v_l_plus_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_plus_phi_prime["aTGC:la_up"].Draw("same")
c_l_plus_phi_prime.Draw()